# Artificial HTS

A notebook to construct artificial HTS samples based on the abundance information extracted from real HTS samples.

Dependencies:
- [lmdbm](https://pypi.org/project/lmdbm/)
- [NCBI Command Line Tools](https://www.ncbi.nlm.nih.gov/datasets/docs/v1/download-and-install/)

In [185]:
from collections import defaultdict
import json
from lmdbm import Lmdb
import matplotlib.pyplot as plt
import multiprocessing as mp
import numpy as np
import os
import re
import subprocess
import zipfile

In [2]:
SEQ_LEN = 150

## Reading and Parsing

The following section reads the abundance information and constructs taxonomy trees for each sample. Each sample tree gives us an idea of what microorganisms are present and how we need to distribute the genomes.

In [3]:
CLASSES = "kpcofgs"

class TaxonomyNode:
    def __init__(self, name, level, abundance=0, parent=None):
        self.name = name
        self.abundance = abundance
        self.level = level
        self.parent = parent
        self.children = {}
        self.accession = None
        self.key = f"{CLASSES[self.level]}__{self.name}"
        self.full_name = self.__build_full_name()

    def path(self):
        path = [self.name]
        node = self.parent
        while node is not None:
            path.append(node.name)
            node = node.parent
        return list(reversed(path))

    def __build_full_name(self):
        prefix = ""
        if self.parent is not None:
            prefix += self.parent.full_name + ';'
        return f"{prefix}{self.key}"

    def __contains__(self, key):
        return key in self.children

    def __getitem__(self, key):
        return self.children[key]

    def __setitem__(self, key, value):
        self.children[key] = value
        return self

    def __str__(self):
        return self.full_name + f"({self.abundance})"

    def __repr__(self):
        return str(self)

In [4]:
class SampleTaxonomy:
    def __init__(self):
        self.children = {}
        self.abundance = 0

    def add(self, tax_classes, abundance=0):
        node = self
        parent = None
        for level, c in enumerate(tax_classes):
            if c not in node:
                node[c] = TaxonomyNode(c, level, parent=parent)
            parent = node[c]
            node = node[c]
        node.abundance += abundance
        return node

    def __contains__(self, key):
        return key in self.children

    def __getitem__(self, key):
        return self.children[key]

    def __setitem__(self, key, value):
        self.children[key] = value
        return self

    def __str__(self):
        for c in self.children.values():
            stack = [(c, 0)]
            while len(stack) > 0:
                node, depth = stack.pop()
                print("\t"*depth, node.key, sep="")
                for c in node.children.values():
                    stack.append((c, depth + 1))

    def __repr__(self):
        return str(self)

In [5]:
def parse_taxonomy_string(taxonomy):
    return re.findall(rf"(?<=[{CLASSES}]__)\[?([a-z\-0-9]+)\]?(?=;|$)", taxonomy.lower())

def read_abundances(path):
    abundances = []
    samples = []
    with open(path) as f:
        taxonomy = f.readline().split(',')[1:]
        for line in f:
            sample, *abundance = line.split(',')
            samples.append(sample)
            abundances.append(list(map(float, abundance)))
    samples = np.array(samples)
    abundances = np.array(abundances).astype(int)
    return samples, taxonomy, abundances

def create_sample_tree(sample, taxonomy, abundance):
    tree = SampleTaxonomy()
    for j in range(len(abundance)):
        if abundance[j] == 0.0:
            continue
        tax = parse_taxonomy_string(taxonomy[j])
        tree.add(tax, abundance[j])
    return tree

In [200]:
samples, taxonomy, abundances = read_abundances("/home/data2/dna/exported/level-7.csv")

In [171]:
sample_trees = [create_sample_tree(s, taxonomy, a) for s, a in zip(samples, abundances)]

---
## Data Fetching

Now that we have the microorganisms to search for, we can pull the genomic data from NCBI.

In [8]:
# 0 = kingdom
# 1 = phylum
# 2 = order
# 3 = class
# 4 = family
# 5 = genus
# 6 = species
MIN_LEVEL = 4

In [170]:
SUMMARY_CACHE_PATH = "/home/data2/dna/cache/summary"
DATA_CACHE_PATH = "/home/data2/dna/cache/data"
OUTPUT_PATH = "/home/data2/dna/synthetic_nachusa"

In [132]:
summary_cache = set([os.path.splitext(f)[0] for f in os.listdir(SUMMARY_CACHE_PATH)])
data_cache = set([os.path.splitext(f)[0] for f in os.listdir(DATA_CACHE_PATH) if f.endswith(".zip")])

In [133]:
def run(cmd):
    result = subprocess.run(cmd, capture_output=True)
    return result.returncode, result.stdout.decode()

In [143]:
def fetch_summary(taxonomy, force=False):
    if taxonomy in summary_cache and not force:
        return taxonomy
    path = os.path.join(SUMMARY_CACHE_PATH, taxonomy + ".json")
    # summary_cache.add(taxonomy)
    exitcode, output = run(["datasets", "summary", "genome", "taxon", taxonomy])
    with open(path, 'w') as f:
        f.write(output)
    return taxonomy

def fetch_summaries(taxonomy, min_level=MIN_LEVEL, num_workers=5):
    tax_to_fetch = set()
    for tax in taxonomy:
        tax_classes = parse_taxonomy_string(tax)
        if len(tax_classes) - 1 < MIN_LEVEL:
            continue
        tax_to_fetch.add(tax_classes[:6][-1])
    with mp.Pool(num_workers) as p:
        taxons = p.map(fetch_summary, list(tax_to_fetch))
    for tax in taxons:
        if tax is None:
            continue
        summary_cache.add(tax)

## Fetch Summaries

Here, we attempt fetch the summary information of each known organism from NCBI. These summaries are cached on the system for future use.

In [135]:
fetch_summaries(taxonomy)

## Load Genome References from Summaries

Since the summaries are quite large, we can extract the genome references to obtain much smaller summaries that we can store in memory.

In [14]:
genome_lookup = {}
for i, tax in enumerate(summary_cache):
    print(f"\r{i+1}/{len(summary_cache)}", end="")
    with open(os.path.join(SUMMARY_CACHE_PATH, tax + ".json")) as in_f:
        summary = json.load(in_f)
    genome_lookup[tax] =  []
    if summary["total_count"] == 0:
        continue
    for a in summary["assemblies"]:
        assembly = a["assembly"]
        if "biosample" not in assembly:
            continue
        # if assembly["assembly_accession"] == accession:
        #     test_assembly = assembly
        genome_lookup[tax].append({
            "accession": assembly["assembly_accession"],
            "organism": assembly["biosample"]["description"]["organism"]["organism_name"].lower()
        })

655/655

## Find Genome Accessions

Here we will locate genome accessions for the species we've found in the samples.

In [15]:
def inject_accessions(tree, rng):
    for c in tree.children.values():
        stack = [(c, 0)]
        while len(stack) > 0:
            node, depth = stack.pop()
            for c in node.children.values():
                stack.append((c, depth + 1))
            if depth < 6:
                continue
            if node.accession is not None:
                continue
            genomes = genome_lookup[node.parent.name]
            choices = [g for g in genomes if node.name in g["organism"]]
            if len(choices) == 0:
                continue
            node.accession = rng.choice(choices)["accession"]

In [16]:
rng = np.random.default_rng(0)
for tree in sample_trees:
    inject_accessions(tree, rng)

## Populate Samples with Random Species

When we have classified down to a particular family/genus, but did not obtain any specific species, we draw a species at random from the same family/genus to use.

In [17]:
def populate_random_species(tree, rng):
    to_add = []
    for c in tree.children.values():
        stack = [(c, 0)]
        while len(stack) > 0:
            node, depth = stack.pop()
            for c in node.children.values():
                stack.append((c, depth + 1))
            if depth < MIN_LEVEL or depth == 6 or len(node.children) > 0:
                continue
            # Family/genus that doesn't have specific species.
            # Select randomly from the summary
            if len(genome_lookup[node.name]) == 0:
                continue
            tax = rng.choice(genome_lookup[node.name])
            try:
                tax_class = re.findall(r"([^\s]+)\s(.*)", tax["organism"])[0]
            except Exception as e:
                continue
            tax_group = node.path() + list(tax_class[depth == 5:])
            assert len(tax_group) == 7

            to_add.append((tax_group, tax["accession"]))
    for tax, accession in to_add:
        node = tree.add(tax)
        node.accession = accession

In [18]:
rng = np.random.default_rng(1)
for tree in sample_trees:
    populate_random_species(tree, rng)

## Tree Pruning

Since there are some genomes that we may not be able to access, we need to prune the trees.

In [19]:
def prune_tree(node, depth=-1):
    if depth == 6:
        return node.accession is None
    prune = True
    to_prune = []
    for child in node.children.values():
        should_prune = prune_tree(child, depth + 1)
        prune &= should_prune
        if should_prune:
            to_prune.append(child.name)
    for key in to_prune:
        del node.children[key]
    if depth == -1:
        return
    return prune

In [20]:
for tree in sample_trees:
    prune_tree(tree)

## Download Genomes

In [152]:
def fetch_genome(accession, force=False):
    path = os.path.join(DATA_CACHE_PATH, accession + ".zip")
    if accession in data_cache and not force:
        return True
    exitcode, output = run(["datasets", "download", "genome", "accession", accession, "--filename", path])
    data_cache.add(accession)
    return exitcode == 0

def fetch_genomes(trees, num_workers=5):
    accessions = set()
    for tree in trees:
        for c in tree.children.values():
            stack = [(c, 0)]
            while len(stack) > 0:
                node, depth = stack.pop()
                for c in node.children.values():
                    stack.append((c, depth + 1))
                if depth < 6:
                    continue
                accessions.add(node.accession)
    with mp.Pool(num_workers) as p:
        p.map(fetch_genome, accessions)

In [137]:
fetch_genomes(sample_trees)

## Abundance Adjustment

Now we need to propogate the abundance values down to the chidlren to obtain the correct number of genomes to use for each species.

In [23]:
def count_species(root):
    root.num_species = root.abundance
    for node in root.children.values():
        root.num_species += count_species(node)
    return root.num_species

def adjust_abundance(root):
    for node in root.children.values():
        if node.num_species == 0:
            assert len(root.children) == 1
            node.abundance += root.abundance
        else:
            node.abundance += int(np.round(node.abundance / node.num_species * root.abundance))
        adjust_abundance(node)
    if len(root.children) > 0:
        root.abundance = 0

In [24]:
for tree in sample_trees:
    count = count_species(tree)
    adjust_abundance(tree)
    # print(count, count_species(tree))

85899 164399
43640 64290
41860 53858
64390 101839
65946 96236
34750 55859
96388 159692
39343 57541
60271 94600
90508 184125
66244 107941
76184 160292
48280 61117
40989 62868
34295 51217
32 32
73061 126381
55789 99972
18278 20675
50925 87597
49519 69330
83634 159806
35498 44984
66456 129596
59382 102748
73942 120808
88422 181506
60235 94675
82221 148157
70299 126645
78878 153075
61831 131700
53959 88964
62226 108008
57445 121401
47500 66623
113937 257361
631 560
93844 136253
51758 85646
34951 42288
44592 69531
41688 69850
66258 113410
42722 62611
110638 203968
85285 131484
55603 79713
80525 172684
57975 107856
154 154
44909 66182
55879 97126
7873 7216
9586 7712
8615 9083
5724 4862
8441 10598
8475 8570
8438 9604
10217 13444
8391 7874
10685 10604
11006 10300
6910 5859
9114 9964
9253 8564
9140 8654
9226 10274
8735 9764
9726 10158
13212 13026
10079 10410
9209 11448
8732 8920
6723 7292
9424 9252
5998 5963
7274 6561
8914 8358
7087 8215
7772 7772
9884 10269
9127 9502
8450 9579
8784 9387
9506 9

## Synthetic HTS Construction

With the polished trees, we can now pull from the downloaded genome data and construct synthetic HTS samples.

In [25]:
def extract_species_from_tree(tree):
    result = []
    for c in tree.children.values():
        stack = [c]
        while len(stack) > 0:
            node = stack.pop()
            if node.level == 6:
                result.append(node)
                continue
            for c in node.children.values():
                stack.append(c)
    return result

In [112]:
genome_cache = {}

In [154]:
def read_genome(accession):
    if accession in genome_cache:
        return genome_cache[accession]
    try:
        archive = zipfile.ZipFile(os.path.join(DATA_CACHE_PATH, accession + ".zip"))
    except (FileNotFoundError, zipfile.BadZipFile) as e:
        print("Redownloading accession:", accession)
        fetch_genome(accession, force=True)
        archive = zipfile.ZipFile(os.path.join(DATA_CACHE_PATH, accession + ".zip"))
    for filename in archive.namelist():
        if os.path.basename(filename).startswith(accession) and filename.endswith(".fna"):
            break
    raw_genome = archive.read(filename).decode()
    archive.close()
    parts = re.split(r"\>.*\n", raw_genome)[1:]
    parts = [part.replace('\n', '') for part in parts]
    valid_parts = []
    for i, part in enumerate(parts):
        match = re.match(r"^[ACTGN]*$", part)
        if match is None or len(part) < SEQ_LEN:
            # chars = set([c for c in part])
            # print(f"Error {i+1}/{len(parts)}", chars)
            continue
        valid_parts.append(sequence_to_bytes(part))
    genome_cache[accession] = valid_parts
    if len(valid_parts) == 0:
        print(f"Warning: Invalid genome {accession}... This genome will be ignored.")
    return valid_parts

In [161]:
def build_hts_sample(tree, rng):
    sequences = []
    labels = []
    species = extract_species_from_tree(tree)
    for i, organism in enumerate(species):
        # print(f"\r{i+1}/{len(species)}", end="")
        genomes = read_genome(organism.accession)
        if len(genomes) == 0:
            continue
        for j in range(organism.abundance):
            # print(f"\r{i+1}/{len(species)}; {j+1:04d}/{organism.abundance:04d}", end="")
            genome = genomes[rng.integers(len(genomes))]
            pos = rng.integers(len(genome) - SEQ_LEN + 1)
            sequences.append(genome[pos:pos+SEQ_LEN])
            labels.append(organism.accession)
    permutation = rng.permutation(len(sequences))
    return np.array(sequences)[permutation], np.array(labels)[permutation]

In [156]:
BASE_TO_I = {b: i for i, b in enumerate("ACTGN")}
def sequence_to_bytes(sequence):
    return np.array([BASE_TO_I[c] for c in sequence], dtype=np.uint8).tobytes()

In [163]:
rng = np.random.default_rng(2)
synthetic_samples = []
for i, tree in enumerate(sample_trees):
    print(f"\r{i+1}/{len(sample_trees)}", end="")
    synthetic_samples.append(build_hts_sample(tree, rng))

206/210Warning: Invalid genome GCA_014844415.1... This genome will be ignored.
209/210Warning: Invalid genome GCA_002356755.1... This genome will be ignored.
210/210

Lastly, we need to convert the accessions to an integer identifier.

In [164]:
genome_id_map = {}
for (_, labels) in synthetic_samples:
    for accession in labels:
        if accession not in genome_id_map:
            genome_id_map[accession] = len(genome_id_map)

In [168]:
final_synthetic_samples = []
for (sample, labels) in synthetic_samples:
    labels = [genome_id_map[key] for key in labels]
    final_synthetic_samples.append((sample, labels))

## Write to LMDB Dictionaries

In [199]:
for i, (sample_name, (sample, labels)) in enumerate(zip(samples, final_synthetic_samples)):
    print(f"\r{i+1}/{len(samples)}", end="")
    to_write = {}
    for i in range(len(sample)):
        to_write[str(f"{i}_s").encode()] = sample[i]
        to_write[str(f"{i}_l").encode()] = bytes(labels[i])
    with Lmdb.open(os.path.join(OUTPUT_PATH, sample_name + ".db"), 'c') as store:
        store.update(to_write)

210/210